# Everything about Unix/Linux they didn't teach you 

In this section, I want to talk about some basic setup in order to interact with an HPC system.

## Learning Objectives

After reading this chapter, you should be able to:

- **Setup** your terminal to connect to an HPC system
- **Navigate** the HPC filesystem using both *absolute* and *relative* paths
- **Find** and **set** environment variables in an HPC system
- **Modify** your `$PATH` variable to include the path to an executable
- **Use** `which` to identify which version of an executable you're using
- **Articulate** basic HPC architecture
- **Identify** the shared filesystems associated with your cluster

## Terminal setup

If you are on Linux/Mac, you'll be working with the terminal. On Windows, you'll need a terminal program such as [PuTTY]() to connect to the remote servers.

In our examples, we're going to be connecting to the Fred Hutch servers `rhino` and the associated cluster, `gizmo`. 

:::{.callout-tip}
## Don't Forget the VPN!

Many HPC systems are behind an organization's VPN, so you'll need a VPN client like Cisco Secure Client to get into your VPN.

FH Users: you'll connect to `rhino` to gain access to the HPC system. 
:::

:::{.callout-note}
## A Little More Advanced
If you are on Windows, you can install [Windows Subsystem for Linux](https://learn.microsoft.com/en-us/windows/wsl/install), and specifically the Ubuntu distribution. That will give you a command-line shell that you can use to interact with the remote server. I prefer this route, but PuTTY works great as well.
:::

On your machine, I recommend using a text editor to edit the scripts in your remote shell. Good ones include [Visual Studio Code (VS Code) ](https://code.visualstudio.com/), or built in ones such as `nano`. You can use VSCode to edit scripts remotely using the SSH extension. Editing scripts remotely like this may be more comfortable for you.

## Where Am I?

One of the confusing things about working on HPC is that sometimes you have a shell open on the head node, but oftentimes, you are on a worker node. 

Your totem for telling which node you're in is `hostname`, which will give you the host name of the machine you're on.

For example, if I used `grabnode` to grab a `gizmo` node for interactive work, I can check which node I'm in by using:

```bash
hostname
```

```
gizmok164
```
If you're confused about which node you're in, remember `hostname`. It will save you from making mistakes.

## Navigating the Unix/Linux filesystem

We'll start out our Unix/Linux journey talking about the filesystem, which has some quirks we need to be aware of.

### Going `/home`: `~/`

There is one important shortcut you should always remember: `~/` is shorthand for your own home directory.

Depending on the linux distribution, this can be a different location. On the FH filesystem, when I use `~/`, it maps to:

`/home/tladera2`

The home directory is also important because it is where your configuration files live, such as `.bashrc` (see below).

### Absolute versus relative paths

Absolute paths contain all the information needed to find a file in a file system from the root `/` directory. For example, this would be an absolute path:

```
/fh/fast/laderas_t/immuno_project/raw_data/tcr_data.fasta
```

Absolute paths always start with `/`, because that is the root directory.

Relative paths break up an absolute path into two pieces of information: 1) your current directory and 2) the path *relative* to that directory. Relative paths are really helpful because things don't break when you move your folder or files. 

If my current path the directory `/fh/fast/laderas_t/immuno_project/`, then the relative path to that same file would be:

```
raw_data/tcr_data.fasta
```

Note that this relative path does not start with a `/`, because our current directory isn't the root directory. 

You may have seen relative paths such as `../another_directory/` - the `..` means to go up a directory in the file hierarchy, and then look in that directory for the `another_directory/` directory. I try to avoid using relative paths like these.

In general for portability and reproducibility, you will want to use relative paths **within a directory**, and avoid using relative paths like `../../my_folder`, where you are navigating up. In general, use relative paths to navigate down. 

### Keep Everything in a Folder

We need to talk about code and data organization. For the FH system, we have a `/home/` directory, and if we have generated research data, a `/fh/fast/` directory.

Ideally, we want to make the naming conventions of our code and our data as similar as possible.

The requirements of bioinformatics analysis usually require a huge amount of data, so storing data and scripts in a single directory is usually not recommended.

### Moving Things Around

A lot of the time, we need to move files between shared filesystems. One filesystem might be good at storage and be backed up on a regular basis, while another filesystem might be better for temporary work on the cluster. 

You're probably familiar with `mv`, which lets you move files around in Unix. One thing to keep in mind when you're `mv`ing things to a new folder that there is a difference between:

```bash
mv log.txt my_folder   ## renames log.txt to my_folder
```

and

```bash
mv log.txt my_folder/  ## moves log.txt to be in my_folder
```
This is one thing that trips me up all the time. 

This is one situation where using a GUI such as Motuz can be very helpful. 

### Things I always forget: the difference between `/home/mydir/`  and `home/mydir/`

Some things that trip me up all the time. The difference between 

```bash
/home/mydir/    #absolute path
```
and 

```bash
home/mydir/`    #relative path
```
The first one is an *absolute path*, and the second is a *relative path*. Your clue is the leading `/` at the beginning of a path. If you're getting `file not found` messages, check to make sure the path is the right format.

## Environment Variables

Environment variables are variables which can be seen globally in the Linux system across executables. 

You can get a list of all set environment variables by using the `env` command. Here's an example from my own system:

```bash
env
```
```
SHELL=/bin/bash
NVM_INC=/home/tladera2/.nvm/versions/node/v21.7.1/include/node
WSL_DISTRO_NAME=Ubuntu
NAME=2QM6TV3
PWD=/home/tladera2
LOGNAME=tladera2
[....]
```

One common environment variable you may have seen is `$JAVA_HOME`, which is used to find the Java Software Development Kit (SDK). (I usually encounter it when a software application yells at me when I haven't set it.)

You can see whether an environment variable is set using `echo`, such as 

```bash
echo $PATH
```

```
/home/tladera2/.local/bin:/home/tladera2/gems/bin:/home/tladera2/.nvm/versions/node/v21.7.1/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/ [....]
```

### Setting Environment Variables

In Bash, we use the `export` command to declare an environment variable. For example, if we wanted to declare the environment variable `$SAMTOOLS_PATH` we'd do the following:

```bash
export SAMTOOLS_PATH="/home/tladera2/miniconda/bin/"
```

One thing to note is that spacing matters when you declare environment variables. For example, this won't declare the `$SAMTOOLS_PATH` variable:

```bash
export SAMTOOLS_PATH = "/home/tladera2/miniconda/bin/"
```

Another thing to note is that we declare environment variables differently than we use then. If we wanted to use `$SAMTOOLS_PATH` we'd use a dollar sign (`$`) in front of it:

```bash
#!/bin/bash/
$SAMTOOLS_PATH/samtools view -c $input_file
```
In this case, the value of `$SAMTOOLS_PATH` will be expanded (substituted) to give:

```bash
#!/bin/bash/
/home/tladera2/miniconda/bin/samtools view -c $input_file
```

### A Very Special Environment Variable: `$PATH`

The most important environment variable is the `$PATH` variable. This variable is important because it determines where to search for software executables. 

We add to the `$PATH` by appending to it. You might have seen the following bit of code in your `.bashrc`:

```bash
export PATH=$PATH:/home/tladera2/samtools/
```

We are adding the path `/home/tladera2/samtools/` to our `$PATH` environment variable. Note that how we refer to the `PATH` variable is different depending on which side the variable is on of the equals sign. 

We declare the variable using `export PATH` and we append to the variable using `$PATH`. This is something that trips me up all the time. 

:::{.callout-note}
## For FH Users

In general, when you use environment modules on `gizmo`, you do not need to modify your `$PATH` variable. You mostly need to modify it when you are compiling executables so that the system can find them. Be sure to use `which` to see where the environment module is actually located:

`which samtools`
:::

### Making your own environment variables

One of the difficulties with working on a cluster is that your scripts may be in one filesystem (`/home/`), and your data might be in another filesystem (`/fh/fast/`). And it might be recommended that you transfer over files to a faster-access filesystem (`/fh/temp/`) to process them.

You can set your own environment variables for use in your own scripts. For example, we might define a `$TCR_FILE_HOME` variable:

```
export TCR_FILE_HOME=/fh/fast/my_tcr_project/
```
to save us some typing across our scripts. We can use this new environment variable like any other existing environment variable:

```bash
#!/bin/Bash
export my_file_location=$TCR_FILE_HOME/fasta_files/
```

### `.bashrc` versus `.bash_profile`

Ok, what's the difference between `.bashrc` and `.bash_profile`?

The main difference is when these two files are sourced. `bash_profile` is used when you do an interactive login, and `.bashrc` is used for non-interactive shells.

`.bashrc` should contain the environment variables that you use all the time, such as `$PATH` and `$JAVA_HOME` for example. You can get the best of both worlds by including the following line in your `.bash_profile`:

```bash
source ~/.bashrc
```

## Working with Shell Scripts

> Note that I'm only covering bash scripting (hence the name of the book). Each shell has different conventions.

When you are writing shell scripts, there's a few things to know to make them executable. 

### The she-bang: `#!`

If you've looked at a shell script and seen the following:

```bash
#| filename: samcount.sh
#!/bin/bash
samtools view -c $1 > $1.counts.txt
```

the `#!` is known as a she-bang - it's a signal to Linux what shell interpreter to use when running the script on the command line.

### Making things executable: `chmod`

Now we have our shell script, we will need to make it executable. We can do this using `chmod`

```bash
chmod +x samcount.sh
```

Now we can run it using:

```bash
./samcount.sh bam_file.bam
```

Because the script is not on our `$PATH`, then we need to specify the location of the script using `./`. 

Note that you can always execute scripts using the `bash` command, even if they're not executable for you on your filesystem. You will still need read access.

```bash
bash samcount.sh bam_file.bam
```
Much more info about file permissions is here: [Permissions (at the Carpentries)](https://carpentries-incubator.github.io/shell-extras/04-permissions/)

### User Access: Groups

The groups that you are a member of essentially control access to other files that you don't own.

You can see which groups you are a member of by using `groups`. For example, on my local Windows Subsystem for Linux filesystem, I am a member of the following groups.

```bash
groups
```

```
tladera2 adm dialout cdrom floppy sudo audio dip video plugdev netdev
```

As an HPC user, you will usually not have root-level access to the cluster. Again, because it is a shared resource, this is a good thing. The trick is knowing how to install software and add it to your path, or run software containers with new software on a shared system.

:::{.callout-note}
## Why Singularity and Not Docker

When we talk more about software environments, we'll talk about Docker.

Docker requires root-level access to run processes on a machine. There is a special `docker` group that has pretty much root-level access. 

On a shared system such as an HPC cluster, we don't want to grant such access to individual users.

Enter [Singularity](https://sylabs.io/docs/), which was designed for HPC clusters from the ground up. You can run Docker/Singularity containers on a shared system without needing root-level access. 
:::

## Useful Utilities

The following section outlines some useful unix utilities that can be very helpful when you're working in bash. Most of these should be available in HPC systems by default.

### Text editors: `vim` or `nano`

In general, we recommend connecting an editor such as VS Code via SSH to make it easier to edit files. But sometimes you just need to edit a file on the system directly.

That's what `nano` and `vim` are for. Of these, `nano` has the smallest learning curve, since it works like most editors. `vim` is powerful, but there is a steep learning curve associated with it. 

### `screen` or `tmux`: keep your session open


{{< video https://youtu.be/fYWszwc-LXM >}}



Oftentimes, when you are running something interactive on a system, you'll have to leave your shell open. Otherwise, your running job will terminate.

You can use `screen` or `tmux`, which are known as window managers, to keep your sessions open on a remote machine. We'll talk about `screen`.

![](images/start_screen.png)

`screen` works by starting a new bash shell. You can tell this because your bash prompt will change.

The key of working remotely with `screen` is that you can then request an `hpc` node. 

For FH users, you can request a `gizmo` node using `grabnode`. We can then check we're on the `gizmo` node by using `hostname`.

If we have something running on this node, we can keep it running by *detaching* the screen session. Once we are detached, we should check that we're back in `rhino` by using `hostname`. Now we can log out and our job will keep running.

![](images/reattach_screen.png)

If we need to get back into that `screen` session, we can use: 

```bash
screen -ls
```
To list the number of sessions:

```
There is a screen on:
        37096.pts-321.rhino01   (05/10/2024 10:21:54 AM)        (Detached)
1 Socket in /run/screen/S-tladera2.
```

 Once we've found the id for our screen session (in this case it's `37096`), we can *reattach* to the screen session using:

 ```
screen -r 37096
 ```
And we'll be back in our screen session! Handy, right?

:::{.callout-note}
## For FH Users

Note that if you logout from `rhino`, you'll need to log back into the same `rhino` node to access your screen session.

For example, if my screen session was on `rhino01`, I'd need to ssh back into `rhino01`, not `rhino02` or `rhino03`.
:::

### The `Tab` key

Never underestimate the usefulness of the tab key, which triggers autocompletion on the command line. It can help you complete paths to files and save you a lot of typing.

### `squeue -u <username>`

Sometimes you will want to know where you are in the queue of all the other jobs that are in the run queue in SLURM. You can use `squeue` with `-u` (username) option to look for your username. For example:


```{bash}
#| eval: false
squeue -u tladera2
```